## PROBLEM 1

In [ ]:
#Code from class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

from zipfile import ZipFile
import os
import statistics

In [ ]:
with ZipFile("PSYC027_INTEGRATE_data.zip", 'r') as zip:
    #zip.printdir() 
  
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

In [ ]:
z = zip.namelist()
z = z[1:]
#print(z)

In [ ]:
data2 = pd.read_csv("PSYC027_INTEGRATE_data/data_S05_INTEGRATE_fMRI1_Retrieval.txt",sep='\\t')
data2.head()

In [ ]:
data2.RT.head()

In [ ]:
data2.Accuracy.head()

In [ ]:
for subject in z:
    #code from class
    #print(subject)
    data = pd.read_csv(subject,sep='\\t')
    p = data.head()
    #print(p)
    #print(data.PairType)

In [ ]:
mu = np.mean(data.RT)
sd = np.std(data.RT)

In [ ]:
# Priming analysis

# Boundary vs NonBoundary: data.BoundaryType == 'B'
# Consecutive items only: data.PairType=='P23'
# data.PrimeType=='P2' - data.PrimeType=='P1'
# data.Accuracy==1
# zscore < nSDs (5)

    
priming_b = []
priming_nb = []
priming_baseline = []
nSDs = 5

In [ ]:
for subject in z:
    #code from class
    print(subject)
    data = pd.read_csv(subject,sep='\\t')
    #print(data.head())
    zscores = st.zscore(data.RT)
    
    #Initiatilize new lists for each subject to calculate average
    #this average would then be appended to priming_b
    priming_b_bysubject = []
    priming_nb_bysubject = []
    priming_baseline_bysubject = []


    for ctr,iRT in enumerate(data.RT):
        #print(ctr)

        # if it's the first trial, pass
        if ctr==0:
            pass
        else:
            # Check that the current trial satisfies our criteria
            #  First check is PrimeType=='P2'
            #AND
            # Consecutive items only: data.PairType=='P23'
            
            
            #condition 1
            validcurrent = data.PairType[ctr]=='P23' and data.PrimeType[ctr]=='P2' and \
                data.Accuracy[ctr]==1 and zscores[ctr] < nSDs

            #condition 2
            # data.PrimeType=='P2' - data.PrimeType=='P1'
            validprev = data.PairType[ctr-1]=='P23' and data.PrimeType[ctr-1]=='P1' and \
                data.Accuracy[ctr-1]==1 and zscores[ctr-1] < nSDs

            #condition 3
            if validcurrent & validprev:
                if data.BoundaryType[ctr]=='B' and data.BoundaryType[ctr-1]=='B':
                    priming_b_bysubject = np.append(priming_b_bysubject,data.RT[ctr]-data.RT[ctr-1])
                    
                elif data.BoundaryType[ctr]=='NB' and data.BoundaryType[ctr-1]=='NB':
                    priming_nb_bysubject = np.append(priming_nb_bysubject,data.RT[ctr]-data.RT[ctr-1])
                    
                    
             #condition 4
            else:
                validbaseline = data.ItemOldNew[ctr]=='old' and \
                    data.ItemOldNew[ctr-1]=='old' and \
                    data.Accuracy[ctr]==1 and data.Accuracy[ctr-1]==1 and \
                    zscores[ctr] < nSDs and zscores[ctr-1] < nSDs
                if validbaseline:
                    priming_baseline_bysubject = np.append(priming_baseline_bysubject,data.RT[ctr]-data.RT[ctr-1])
                    
            #calculate averages and append
    a = np.nanmean(priming_b_bysubject)
    b = np.nanmean(priming_nb_bysubject)
    c = np.nanmean(priming_baseline_bysubject)

    priming_b = np.append(priming_b,a)
    priming_nb = np.append(priming_nb,b)
    priming_baseline = np.append(priming_baseline,c)


In [ ]:
priming_b_bysubject

In [ ]:
priming_b

In [ ]:
print(priming_b.shape)
#print(priming_b)

print(priming_nb.shape)
#print(priming_nb)

print(priming_baseline.shape)
#print(priming_baseline)

In [ ]:
print(np.nanmean(priming_b))
print(np.nanmean(priming_nb))
print(np.nanmean(priming_baseline))

In [ ]:
#creating a graph
m1 = np.nanmean(priming_b)
m2 = np.nanmean(priming_nb)
m3 = np.nanmean(priming_baseline)

objects = ("B", "NB", "Control")
y_pos = np.arange(len(objects))
means = [m1, m2, m3]


In [ ]:
plt.bar(y_pos, means, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('RT (ms)')
plt.xlabel('Condition name')

In [ ]:
#compare 21 and 21 shaped arrays for each subject
print(st.ttest_rel(priming_b,priming_nb))
print(st.ttest_rel(priming_b,priming_baseline))
print(st.ttest_rel(priming_nb,priming_baseline))

### THERE IS A SIGNIFICANT DIFFERENCE IN PRIMING. this still holds while comparing to the baseline condition.
for boundary conditon, the subjects on average recalled the second word FASTER than the first word (hence the positive difference between w1-w2). for the non-boundary conditon, the subhects on average recalled the second word SLOWER than the first word (hence the negative difference between w1-w2). 

In [ ]:
%whos

## PROBLEM 2

In [3]:
from psychopy import visual, core, event
import random
import numpy as np
import pandas as pd
from zipfile import ZipFile

In [4]:
text_file = open("wordpool.txt", "r")
words = text_file.readlines()

In [5]:
words[0:100]
len(words)

1000

In [6]:
def experiment(ntargets,nlures):
    int(ntargets)
    int(nlures)
    lure_indexes = []
    target_indexes = []
    
    for i in range(ntargets):
        r=random.randint(1,len(words))
        target_indexes.append(r)
    
    for i in range(nlures):
        r=random.randint(1,len(words))
        while r in target_indexes:
            r=random.randint(1,len(words))
        lure_indexes.append(r)
    
    print(target_indexes)
    print(lure_indexes)
    
    return target_indexes, lure_indexes

In [7]:
targets, lures = experiment(3, 3)

[812, 215, 58]
[218, 635, 403]


In [8]:
def learning_phase(pres_lenth=1.0):
    win = visual.Window([400,400],pos=[0,0])
    
    for target in targets:
        wd = visual.TextStim(win, text = words[target])
        wd.draw()

        #flip window
        win.flip()
        core.wait(pres_lenth)

    win.close()

In [10]:
lures[2]

403

In [ ]:
learning_phase()

In [ ]:
a = 4
subject_number = 1
subject_arr = np.full((a,),subject_number)
subject_arr = list(subject_arr)

In [ ]:
subject_arr

In [25]:
def test_phase(subject_number):
    import pandas as pd 

    all_indexes = targets + lures
    print(all_indexes)
    random.shuffle(all_indexes)
    print(len(all_indexes))
    
    #creating the first list for our pandas dataframe
    subject_number = int(subject_number)
    subject_repeated = np.full((len(all_indexes),),subject_number)
    subject_repeated = list(subject_repeated) #done
    print(subject_repeated)
    
   
    #creating the rest of the lists for our pandas dataframe
    all_words = [] #done
    status_list = []
    pressedkeys = []
    old_or_new = []
    response_times = []
    confidence_ratings = []
    
    target_results = []
    lure_results = []
    final_results = []
    
    
    #creating the window to present the words
    
    win = visual.Window([400,400],pos=[100,100])
    message = visual.TextStim(win, text = "press j for OLD and k for NEW", pos=[.5,0])
    message.draw()
    win.flip()
    core.wait(2.0)

    
    for word in all_indexes:
        all_words.append(word)
        if word in targets: 
            status_list.append("target")
        else:
            status_list.append("lure")
                
        message = visual.TextStim(win, text = words[word], pos=[.5,0])
        message.draw()
        win.flip()
        
        #adding in stopwatch for timekeeping
        prestime = core.getTime()
        
        #checking for what key was pressed and appending to list
        pk = event.waitKeys(keyList = ["j", "k"],timeStamped=True)
        print(pk) #should be a list of lists
        
        response = str(pk[0][0])
        pressedkeys.append(response)
        response_times.append(pk[0][1]-prestime)

        
        if response == "j":
            old_or_new.append("old")
        else:
            old_or_new.append("new")

        
        #FOR EVEN SUBJECT NUMBERS
        if (subject_number%2) == 0:
            message = visual.TextStim(win, text = "on a scale of 1 to 5, how certain are you? 1= surenew and 5 = sureold", pos=[.5,0])
            message.draw()
            win.flip()
            
            #checking for what confidence was pressed and appending to list
            rating = event.waitKeys(keyList = ["1", "2", "3", "4", "5"])
            print(rating) 
            confidence = str(rating[0])
            confidence_ratings.append(confidence)
        
        
        #FOR ODD SUBJECT NUMBERS
        if (subject_number%2) == 1:
            message = visual.TextStim(win, text = "on a scale of 1 to 5, how certain are you? 1 = sureold and 5 = surenew", pos=[.5,0])
            message.draw()
            win.flip()
            
            rating = event.waitKeys(keyList = ["1", "2", "3", "4", "5"])
            print(rating) 
            confidence = str(rating[0])            
            if confidence == "1":
                confidence = "5"
            elif confidence == "2":
                confidence = "4"
            elif confidence == "4":
                confidence = "2"
            elif confidence == "5":
                confidence = "1"
            
            confidence_ratings.append(confidence)   
            

        
        if word in targets and response == "j":
            final_results.append("hit")
            target_results.append(1) #CORRECT response to a target
        elif word in targets and response == "k":
            final_results.append("miss")
            target_results.append(0) 
            
        if word in lures and response == "k":
            final_results.append("correct_rejection")
            lure_results.append(1)  #CORRECT response to a lure
        

        elif word in lures and response == "j":
            final_results.append("false_alarm")
            lure_results.append(0)
    

    win.close()
    

    df = pd.DataFrame(list(zip(subject_repeated, all_words, status_list, pressedkeys, old_or_new, confidence_ratings, response_times, final_results)), 
               columns =['Subject_Number', 'Word', "Status", "Response_Button", "Old/New", "Confidence", "RespTime", "Response_Type"])
    return df

In [29]:
test_phase_results = test_phase(101)

[812, 215, 58, 218, 635, 403]
6
[101, 101, 101, 101, 101, 101]
[['j', 489.72378878100426]]
['1']
[['j', 490.10852595302276]]
['1']
[['j', 490.48574084902066]]
['1']
[['j', 490.8210552240198]]
['1']
[['j', 491.1204292430193]]
['1']
[['j', 491.4199253420229]]
['1']
485.8287 	WARNING 	Monitor specification not found. Creating a temporary one...


In [30]:
test_phase_results

,Subject_Number,Word,Status,Response_Button,Old/New,Confidence,RespTime,Response_Type
0,101,58,target,j,old,5,1.066025,hit
1,101,403,lure,j,old,5,0.212501,false_alarm
2,101,215,target,j,old,5,0.179335,hit
3,101,812,target,j,old,5,0.148275,hit
4,101,635,lure,j,old,5,0.114222,false_alarm
5,101,218,lure,j,old,5,0.113926,false_alarm


In [ ]:
rts

In [ ]:
m = np.mean(tgt_results)
print("the hit rate is %s" %m) #how many participants pressed 1 correctly in the targets 

In [ ]:
n = 1- np.mean(l_results) #how many participants pressed 0 in the lures
print("the false alarm rate is %s" %n)

In [ ]:
tgt_results

In [ ]:
l_results

In [ ]:
len(tgt_results)

In [ ]:
len(l_results)

In [ ]:
lst = [1,4,6]
lst2 = [5,6]
test_df = pd.DataFrame(list(zip(lst, lst2)), 
               columns =['list1', 'list2'])

In [ ]:
test_df

In [ ]:
test_phase_results.to_csv("all_data.csv")

## PROBLEM 3

In [1]:
from zipfile import ZipFile
import os
#from psychopy import visual, core, event

In [2]:
pwd

'/mnt/saturn/jupyterhub_data/erangan1/Psych_027_HW'

In [ ]:
cd '/Users/eishnar/Documents/Fall 2019/Week 11'

In [10]:
with ZipFile("GC_photos.zip", 'r') as zip:
    zip.printdir() 
  
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 


File Name                                             Modified             Size
GC_photos/                                     2019-12-08 16:27:58            0
GC_photos/panera.jpeg                          2019-07-14 19:12:04        30871
__MACOSX/                                      2019-12-08 16:29:30            0
__MACOSX/GC_photos/                            2019-12-08 16:29:30            0
__MACOSX/GC_photos/._panera.jpeg               2019-07-14 19:12:04          700
GC_photos/.DS_Store                            2019-12-08 16:28:02         6148
__MACOSX/GC_photos/._.DS_Store                 2019-12-08 16:28:02          120
GC_photos/dominos.png                          2019-07-14 19:06:00        93098
__MACOSX/GC_photos/._dominos.png               2019-07-14 19:06:00          665
GC_photos/stabucks_internet.png                2019-07-14 18:26:20       793079
__MACOSX/GC_photos/._stabucks_internet.png     2019-07-14 18:26:20          745
GC_photos/target.jpg                    

In [ ]:
#os.listdir("images.zip")

In [4]:
y = zip.namelist()

In [5]:
y = y[1:]

In [6]:
y

['images/Steve_Carell.jpg',
 'images/Anne_Hathaway.jpg',
 'images/Amy_Poehler.jpg',
 'images/Jay_Z.jpg']

In [ ]:
for item in y:
    print(item)

type(item)

In [ ]:
def images(pres_lenth=1.0):
    
    win = visual.Window([500,500],pos=[200,200])
    #win2 = visual.Window([500,500],pos=[0,0], color=(0, 150, 0))
    
    for item in y:
        print(item)
        stimulus = visual.ImageStim(win,image=item, size = [.75,.75])
        stimulus.draw()

        win.flip()
        core.wait(pres_lenth)
        
        blank = visual.TextStim(win,text="+",pos = [1,0])
        blank.draw()
        win.flip()
        
        #win2.flip()
        core.wait(0.5)

    win.close()
    #win2.close()

In [ ]:
images()